In [1]:
import pandas as pd
import numpy as np
import os

In [8]:
df = pd.DataFrame([])

path = '/Users/rodrigobraga/Downloads/dados_inmet'

yearmonths = sorted([file.split('_')[3].rstrip('.csv') for file in os.listdir(path) if file.startswith('dados')])

In [ ]:
for yearmonth in yearmonths:
    
    print('Processando {}...'.format(yearmonth))
    
    tmp = pd.read_csv(path + '/dados_aut_emp_{}.csv'.format(yearmonth), sep='\t', index_col=0)
    
    tmp['dc_nome'] = tmp['dc_nome'].str.lower()
    
    tmpg = tmp.groupby(['cd_estacao', 'dc_nome', 'vl_latitude', 'vl_longitude', 'dt_medicao']).agg({'chuva':'sum'}).reset_index()
    
    df = df.append(tmpg)

In [10]:
df.to_csv('inmet_emp.csv')

## Monthly data

In [13]:
# create date as datetime

df['date'] = pd.to_datetime(df['dt_medicao'])

# aggregate variables by station and month

dfg = df.groupby([df.cd_estacao,
                  df.dc_nome,
                  df.vl_latitude,
                  df.vl_longitude,
                  df.date.dt.strftime('%Y-%m')]).agg({'chuva':'sum'}).reset_index()

# replace name of index

dfg.rename(columns={'date':'yearmonth'}, inplace=True)

# convert back do datetime

dfg['yearmonth'] = pd.to_datetime(dfg['yearmonth'], format='%Y-%m-%d')

# sort values

dfg.sort_values(['cd_estacao', 'yearmonth'], inplace=True)

# delete df from memory

dfg.to_csv('inmet_memp.csv')